In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Neural Networks Learning with Scikit-Learn

`scikit-learn` is a large, well-documented and widely used library for machine learning. It contains aslo a support for neural networks. Namely two classes 
* `MLPClassifier` is a class implementing classifiers based on multi-layered neural networks, and 
* `MLPRegressor` is a class implementing regressing (i.e., approximating functions) with multilayered neural networks.

## MLPRegressor 

is an estimator available as a part of the neural_network module of `sklearn` for performing regression tasks using a multi-layer perceptron.

Let us start with a simple example in that we will train a neural network to compute an approximation of the function 
$$2 sin(3x+1)+3$$ on the interval $\langle -3, 5 \rangle$.

### Preparing Data
At first we will prepare a training and testing set.

In [ ]:
X = np.arange(-3, 5, 0.2).reshape(-1,1)
y = 2 * np.sin(3 * X + 1) + 3
print("X:",X)
print("y:",y)

Note that we reshape the 1D vector `np.arange(-3, 5, 0.2)` into a 2D array, where each row is an input pattern. `MLPRegressor` expects that both inputs and outputs are 2D arrays.

We can split manually `X` and `y` into a training and test sets, but there is a function `train_test_split` that does this for us.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
n = 4
print(f"The first {n} inputs:")
print(X_test[:n])
print(f"The corresponding {n} labels:")
print(y_test[:n])

The first 4 inputs:
[[8.00000000e-01]
 [2.00000000e-01]
 [2.66453526e-15]
 [2.20000000e+00]]
The corresponding 4 labels:
[[2.4889178 ]
 [4.99914721]
 [4.68294197]
 [4.93583934]]


Using parameter `random_state=42` we can repeat exactly the same splitting arbitrarily.

### Training `MLPRegressor`

Class MLPRegressor implements a multi-layer perceptron (MLP) that trains using backpropagation with identity a the activation function in the output layer. Hence, the output neurons just output their potentials. In this way, such network can output any real value. The error function used by the network is the mean squared error.

MLPRegressor also supports regression with multi-dimensional outputs -  the desired output can be a vector.

In [5]:
from sklearn.neural_network import MLPRegressor

mlp_regressor  = MLPRegressor(random_state=123)
mlp_regressor.fit(X_train, y_train)

C:\ProgramData\Anaconda3\envs\idp3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\envs\idp3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(random_state=123)

Now we can use the learned network to compute the approximation for the test samples.

In [6]:
y_preds = mlp_regressor.predict(X_test)

print(y_preds[:10])
print(y_test[:10])

print('Test R^2 Score : %.3f'%mlp_regressor.score(X_test, y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor.score(X_train, y_train))

[1.75257859 1.67232722 1.69045494 2.32241619 2.81909897 1.83633411
 3.70868981 2.44840648]
[[2.4889178 ]
 [4.99914721]
 [4.68294197]
 [4.93583934]
 [4.26253328]
 [1.56528782]
 [4.99605331]
 [4.88146111]]
Test R^2 Score : -2.017
Training R^2 Score : -0.170


We can see that the network did not learn the function. In the documentation for `MLPRegressor` we can see that this regressor has many parameter with some default values. 

Let us try 500 iteration instead of the default 200, less hidden neurons - 10 instead of 100 in one hidden layer, sigmoid activation function (`'logistic'`) instead of the default `'relu'` and the learning algorithm `'lgfgs'` that is a variant of Newton's method with a limited memory.

In [7]:
mlp_regressor  = MLPRegressor(random_state=123, 
                              hidden_layer_sizes=[10], 
                              max_iter=500, 
                              activation='logistic',
                              solver='lbfgs')
mlp_regressor.fit(X_train, y_train.ravel())

C:\ProgramData\Anaconda3\envs\idp3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPRegressor(activation='logistic', hidden_layer_sizes=[10], max_iter=500,
             random_state=123, solver='lbfgs')

In [8]:
print('Test R^2 Score : %.3f'%mlp_regressor.score(X_test, y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor.score(X_train, y_train))

Test R^2 Score : 0.850
Training R^2 Score : 0.989


In [9]:
XX = np.arange(-7, 10, 0.05).reshape(-1, 1)
plt.plot(XX, 2 * np.sin(3 * XX + 1) + 3, 'r')
plt.plot(X_train,y_train,'ob')
plt.plot(X_test, y_test,'+b')
plt.plot(XX, mlp_regressor.predict(XX),'g')


$R^2$ score is called the *coefficient of determination* of the prediction. It is defined as $(1-\frac{u}{v})$, where $u$ is the residual sum of squares `((y_true - y_pred) ** 2).sum()` and $v$ is the total sum of squares `((y_true - y_true.mean()) ** 2).sum()`. The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). What is the range for $R^2$?

In [10]:
from sklearn.metrics import mean_squared_error

print("Train set MSE:", mean_squared_error(y_train, mlp_regressor.predict(X_train)))
print("Test set MSE:", mean_squared_error(y_test, mlp_regressor.predict(X_test)))

mlp_regressor.coefs_[0]

Train set MSE: 0.019122492202522406
Test set MSE: 0.23123265335267779


array([[  4.728428  ,  -6.15250299, -32.12745557, -12.46407964,
          6.08030228,   7.79743174,   0.46118822,  -1.56870675,
          6.28921798,   0.49718789]])

Implement the following function that plots the evolution of MSE.

**Note:** You can switch-off warnings using

    import warnings
    warnings.filterwarnings("ignore")


In [39]:
def monitor_MSE(regressor, X_train, X_test, y_train, y_test, iter_step=50):
    # Fit the regressor regressor using traing set X_train with
    # the desired outputs y_train, after each iteration compute
    # MSE for the training and test sets.
    # At the end plot the evolution of MSE on both train and test sets.
    import warnings
    warnings.filterwarnings("ignore")
    
    max_iter = regressor.max_iter
    regressor.set_params(max_iter=iter_step, warm_start=True)
    orig_regressor_params = regressor.get_params()
    print(orig_regressor_params)
    
    ...
    
    plt.yscale('log')
    plt.plot(range(0,max_iter,iter_step), MSE_train, 'b')
    plt.plot(range(0,max_iter,iter_step), MSE_test, 'r')
    return regressor

In [12]:
mlp_regressor  = MLPRegressor(random_state=123, 
                              hidden_layer_sizes=[10], 
                              max_iter=5000, 
                              activation='logistic',
                              solver='lbfgs',
                              early_stopping=True)
mlp_regressor = mlp_regressor.fit(X_train, y_train.ravel())

print('Test R^2 Score : %.3f'%mlp_regressor.score(X_test, y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor.score(X_train, y_train))

print("Train set MSE:", mean_squared_error(y_train.ravel(), mlp_regressor.predict(X_train)))
print("Test set MSE:", mean_squared_error(y_test.ravel(), mlp_regressor.predict(X_test)))

print(mlp_regressor.coefs_[0])

print("Now with monitor")

Test R^2 Score : 0.945
Training R^2 Score : 0.998
Train set MSE: 0.004151945985802848
Test set MSE: 0.08424457564174447
[[  3.4340261   -5.58283526 -23.05428228 -11.11398841   4.04043017
    4.82493507  -3.5903247   -0.93185004   4.90198411   0.05615481]]
Now with monitor


In [ ]:
mlp_regressor  = MLPRegressor(random_state=123, 
                              hidden_layer_sizes=[10], 
                              max_iter=500, 
                              activation='logistic',
                              solver='lbfgs',
                              warm_start=True)
mlp_regressor = monitor_MSE(mlp_regressor, X_train, X_test, y_train.ravel(), y_test.ravel(), iter_step=50)

print('Test R^2 Score : %.3f'%mlp_regressor.score(X_test, y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor.score(X_train, y_train))

print("Train set MSE:", mean_squared_error(y_train, mlp_regressor.predict(X_train)))
print("Test set MSE:", mean_squared_error(y_test, mlp_regressor.predict(X_test)))

mlp_regressor.coefs_[0]

In [ ]:
mlp_regressor  = MLPRegressor(random_state=123, 
                              hidden_layer_sizes=[100], 
                              max_iter=1000, 
                              activation='tanh',
                              learning_rate_init=0.0002,
                              solver='adam',
                              warm_start=True)
mlp_regressor = monitor_MSE(mlp_regressor, X_train, X_test, y_train.ravel(), y_test.ravel(), iter_step=50)

print('Test R^2 Score : %.3f'%mlp_regressor.score(X_test, y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor.score(X_train, y_train))

print("Train set MSE:", mean_squared_error(y_train, mlp_regressor.predict(X_train)))
print("Test set MSE:", mean_squared_error(y_test, mlp_regressor.predict(X_test)))

mlp_regressor.coefs_[0]

In [53]:
XX = np.arange(-7, 10, 0.05).reshape(-1, 1)
plt.plot(XX, 2 * np.sin(3 * XX + 1) + 3, 'r')
plt.plot(X_train,y_train,'ob')
plt.plot(X_test, y_test,'+b')
plt.plot(XX, mlp_regressor.predict(XX),'g')


Implement the following function that performs so called regression analysis of the trained regressor.  

{'activation': 'logistic', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': [10], 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 50, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 123, 'shuffle': True, 'solver': 'lbfgs', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': True}
[[  0.06430247   0.25753642 -11.08051036  -3.69227864   4.20105217
    2.17058039   0.22197992   0.24211164   1.89712524   0.28513069]]
[[  0.65294498   0.43073454 -11.06192323  -7.84864784   2.23222653
    1.13688166   1.80552995   1.84482993   1.86827917   1.46081546]]
[[ 2.34278025 -2.30306134 -9.55211489 -5.15619985  5.94067446  1.51549353
   4.25057605  3.61300111  2.30645062  2.00083589]]
[[ 2.2157701  -2.16364863 -6.23638944 -3.76760157  8.08337329  1.04657239
   9.36012967  4.25905764  3.66447917  2.083

array([[ 3.83743724, -0.21994927, -1.3056198 , -5.06370601,  4.31528423,
         5.24721732,  4.18346231,  5.45431211,  4.42956878,  3.0371101 ]])

In [15]:
mlp_regressor  = MLPRegressor(random_state=123, 
                              hidden_layer_sizes=[10], 
                              max_iter=5000, 
                              activation='logistic',
                              solver='lbfgs',
                              warm_start=True)
mlp_regressor = monitor_MSE(mlp_regressor, X_train, X_test, y_train.ravel(), y_test.ravel(), iter_step=50)

print('Test R^2 Score : %.3f'%mlp_regressor.score(X_test, y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor.score(X_train, y_train))

print("Train set MSE:", mean_squared_error(y_train, mlp_regressor.predict(X_train)))
print("Test set MSE:", mean_squared_error(y_test, mlp_regressor.predict(X_test)))

mlp_regressor.coefs_[0]

In [ ]:
def regression_analysis(regressor, X_train, X_test, y_train, y_test):
    # plot two plots: regressor predictions (y-axis) for the train 
    # set X_train against y_train (x-axis)
    # and regressor predictions (y-axis) for the test 
    # set X_test against y_test (x-axis)
    pass


In [19]:
regression_analysis(mlp_regressor, X_train, X_test, y_train.ravel(), y_test.ravel())

## The Main Attributes of `MLPRegressor`

* `loss_` is the loss after training,
* `n_layers_` is the number of layers of the network,
* `coefs_` is a list of arrays of length `n_layers - 1`; $i$-th array is the weight matrix of layer $i$,
* `intercepts_` is a list of arrays of length `n_layers - 1`; $i$-th array is the bias vector associated with layer $i$, 
* `n_iter_` is the number of iterations the solver run,
* `out_activation_` is the name of the output layer activation function.

`MLPRegressor` uses the Square Error loss function that for an input pattern for which the network predicted $y_{pred}$ and the target value is $y_{true}$ computes $$loss(y_{pred}, y_{true},\mathbf{W}) = \frac{1}{2}\lVert y_{pred} - y_{true}\rVert_2^2 + \frac{\alpha}{2}\lVert \mathbf{W} \rVert_2^2.$$ The `loss_` attribute is then the mean value of the loss on the train set.

In [20]:
print("Loss:", mlp_regressor.loss_)

Loss: 0.010434198256274054


In [21]:
print("Number of Coefs - the weight matrices:", len(mlp_regressor.coefs_))

[weights.shape for weights in mlp_regressor.coefs_]

Number of Coefs - the weight matrices: 2


[(1, 10), (10, 1)]

In [22]:
print("Number of Intercepts - the number of vectors with biases:", len(mlp_regressor.intercepts_))

[intercept.shape for intercept in mlp_regressor.intercepts_]

Number of Intercepts - the number of vectors with biases: 2


[(10,), (1,)]

In [23]:
print("Number of Iterations for Which Estimator Ran:", mlp_regressor.n_iter_)

Number of Iterations for Which Estimator Ran: 50


In [24]:
print("Name of Output Layer Activation Function:", mlp_regressor.out_activation_)

Name of Output Layer Activation Function: identity


`MLPRegressor` has much more parameters. for a complete list see the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor). We will use some of them:

* `hidden_layer_sizes` is a tuple of integers specifying sizes of hidden layers in multi layer perceptrons with default=(100,),
* `activation` specifies activation function for hidden layers. It accepts one of below strings as input. Default value is `relu`:
  * `'identity'` - No Activation. $f(x) = x$,
  * `'logistic'` - Logistic Sigmoid Function. $f(x) = \frac{1}{1 + e^-x}$,
  * `'tanh'` - Hyperbolic tangent function. $f(x) = \tanh(x)$,
  * `'relu'` - Rectified Linear Unit function. $f(x) = \max(0, x)$,

* `solver` accepts one of `'lbfgs'`, `'sgd'`, `'adam'` strings specifying which optimization solver to use for updating weights of neural network hidden layer perceptrons. default=`'adam'`,

* `learning_rate_init` specifies initial learning rate to be used. Based on value of this parameter weights of perceptrons are updated. default=`0.001`,
* `learning_rate` specifies learning rate schedule to be used for training. It accepts one of below strings as value and only applicable when solver='sgd'.
  * `'constant'` - Keeps learning rate constant through a learning process which was set in `learning_rate_init`.
  * `'invscaling'` - It gradually decreases learning rate. `effective_learning_rate = learning_rate_init / pow(t, power_t)`,
  * `'adaptive'` - It keeps learning rate constant as long as loss is decreasing or score is improving. If consecutive epochs fails in decreasing loss according 
    to `tol` parameter and `early_stopping` is on, then it divides current learning rate by 5,
    
* `batch_size` accepts integer value specifying size of batch to use for dataset. Default=`'auto'`. The default auto batch size will set batch size to `min(200, n_samples)`,

* `tol` accepts float values specifying threshold for optimization. When training loss or score is not improved by at least `tol` for `n_iter_no_change` iterations, then optimization ends if `learning_rate` is constant else it decreases learning rate if `learning_rate` is adaptive. Default=`0.0001`,

* `alpha` specifies L2 penalty coefficient to be applied to perceptrons. Default=`0.0001`,

* `momentum` specifies momentum to be used for gradient descent and accepts float value between 0-1. It's applicable when solver is `sgd`,

* `early_stopping` is a boolean value specifying whether to stop training if training score/loss is not improving. Default=`False`,

* `validation_fraction` is a float value between 0-1 specifying amount of training data to keep aside if early_stopping is set. Default=`0.1`.

## Optimizing Model by Grid Search

We have seen that MLPRegressor has many parameters. The search for a suitable 
combination of values for several parameters can be tedious. Let us collect values for
such parameters that we would like to try.

In [25]:
import itertools
from sklearn.model_selection import GridSearchCV

In [26]:
params = {'activation': ['relu', 'tanh', 'logistic'],
          'hidden_layer_sizes': list(itertools.permutations([10,20,50],2)) + [10, 20, 30, 50],
          'solver': ['adam','lbfgs'],
          'learning_rate': ['constant', 'adaptive', 'invscaling']
         }                                

Now `GridSearchCV` can try all combinations of params. For compapring the combinations we will use 5-fold cross-validation.

In [28]:
%%time

mlp_regressor_grid = GridSearchCV(MLPRegressor(random_state=123), param_grid=params, n_jobs=-1, cv=5, verbose=5)
mlp_regressor_grid.fit(X_train,y_train)

print('Train R^2 Score : %.3f'%mlp_regressor_grid.best_estimator_.score(X_train, y_train))
print('Test R^2 Score : %.3f'%mlp_regressor_grid.best_estimator_.score(X_test, y_test))
print('Best R^2 Score Through Grid Search : %.3f'%mlp_regressor_grid.best_score_)
print('Best Parameters : ',mlp_regressor_grid.best_params_)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
Train R^2 Score : 0.938
Test R^2 Score : 0.314
Best R^2 Score Through Grid Search : 0.817
Best Parameters :  {'activation': 'relu', 'hidden_layer_sizes': (10, 50), 'learning_rate': 'constant', 'solver': 'lbfgs'}
Wall time: 21.4 s


In [ ]:
XX = np.arange(-5, 10, 0.05).reshape(-1, 1)
plt.plot(XX, 2 * np.sin(3 * XX + 1) + 3, 'r')
plt.plot(X_train,y_train,'ob')
plt.plot(X_test, y_test,'+b')
plt.plot(XX, mlp_regressor_grid.best_estimator_.predict(XX),'g')

Try to improve the regressor by fine-tuning the initial value for the learning rate - parameter `learning_rate_initr` - and the regularization coefficient `alpha` - L2 penalty parameter and/or different architectures for the network.

In [72]:
params = {'activation': ['relu', 'tanh', 'logistic'],
          'hidden_layer_sizes': [(50,10), (30,5)] + [10, 20, 30, 40, 50],
          'solver': ['adam','lbfgs'],
          'learning_rate': ['constant', 'adaptive', 'invscaling'],
          'learning_rate_init': [0.1, 0.01, 0.001],
          'alpha': [0.00001, 0.0001, 0.001] 
         } 

Can you describe any strategy (heuristics) how be more efficient than the brute-force grid search?

# MLPClassifier

MLPClassifier is a classifier based on multi-layer perceptron. It differs from `MLPRegressor` mainly in the activation function at the output layer - it is logistic (sigmoid) and the output of the network is computed as the sofmax function. For classification, the target class is encoded using one-hot-encoding. For $K$ classes, the value of the class is a vector that contains exactly one value 1 and the remaining elements are zeros. E.g., let $K=3$, then 
1. class will be encoded as $(1,0,0,0)$,
2. class will be encoded as $(0,1,0,0)$,
3. class will be encoded as $(0,0,1,0)$,
4. class will be encoded as $(0,0,0,1)$.

Hence, the output layer will have $K$ neurons - $i$-th neuron corresponds to the $i$-th class. The outputs on the output layer is computes as
$$softmax(\xi_i) = \frac{e^{\xi_i}}{\sum_{j=1}^K e^\xi_j},$$
where $\xi_i$ is the potential at neuron $i$. The predicted class is class $i_0$ such that its corresponding value of $softmax(\xi_i)$ is maximal. We can interpret the output of the $i$-th neuron of the softmax layer as the probability of belonging to class $i$. 

Let us try a simple example. Train data we get directly from `scikit-learn` library.

In [29]:
from sklearn.datasets import load_digits, load_boston

digits = load_digits()
X_digits, y_digits = digits.data, digits.target
print('Dataset Sizes : ', X_digits.shape, y_digits.shape)

Dataset Sizes :  (1797, 64) (1797,)


The description of the data is stored together with the data.

In [ ]:
print(digits.DESCR)

Let us prepare train and test data. In order to have balanced train set, we will used the so-call stratified selection so that the class composition for both train and test sets will be the same. When we put 80% of samples in the train set and 20% in the test set, then for each class, 80% of samples will go into the train set and 20% in the test set.

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, train_size=0.80, test_size=0.20, stratify=y_digits, random_state=123)
print('Train/Test Sizes : ', X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Train/Test Sizes :  (1437, 64) (360, 64) (1437,) (360,)


In [34]:
from sklearn.neural_network import MLPClassifier

mlp_classifier  = MLPClassifier(random_state=123)
mlp_classifier.fit(X_train, y_train)
print(mlp_classifier.get_params())

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 123, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}


## Training and Evaluatin the Classifier

In [35]:
y_preds = mlp_classifier.predict(X_test)

print(y_preds[:15])
print(y_test[:15])

print('Test Accuracy : %.3f'%mlp_classifier.score(X_test, y_test)) ## Score method also evaluates accuracy for classification models.
print('Training Accuracy : %.3f'%mlp_classifier.score(X_train, y_train))

[5 9 9 6 1 6 6 9 8 7 4 2 1 4 3]
[5 9 9 6 1 6 6 9 8 7 4 2 1 4 3]
Test Accuracy : 0.983
Training Accuracy : 1.000


## Confusion Matrix

Confusion matrix is a matrix $C=(c_ij)_{K\times K}$, where $c_ij$ is the number of observations known to be in class $i$ and predicted to be in class $j$. We will use the following function to plot such confusion matrices.

In [36]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_test, y_preds, K):
    # we suppose that y_test and y_preds contain numbers between 0 and K-1
    conf_mat = confusion_matrix(y_test, y_preds)
    fig = plt.figure(figsize=(8,8))
    plt.matshow(conf_mat, cmap=plt.cm.Blues, fignum=1)
    plt.yticks(range(K), range(K))
    plt.xticks(range(K), range(K))
    plt.xlabel('Predicted class')
    plt.ylabel('True class')
    plt.colorbar();
    for i in range(K):
        for j in range(K):
            plt.text(i-0.2,j+0.1, str(conf_mat[j, i]), color='r')

In [37]:
plot_confusion_matrix(y_test, mlp_classifier.predict(X_test), 10)

## The Main Attributes of `MLPClassifier`

The attributes of `MLPClassifier` and `MLPRegressor` are almost the same. The main difference is in the loss function.

`MLPClassifier` uses the cross-entropy loss function. For binary classification if the network predicted $y_{pred}$ and the target value was $y_{true}$, then the loss is
$$loss(y_{pred}, y_{true},\mathbf{W}) = -y_{true} \ln y_{pred} - (1-y_{true}) \ln (1 - y_{pred}) + \frac{\alpha}{2}\lVert \mathbf{W} \rVert_2^2.$$ 

In case of more than two classes, let the true labels for a set of samples be encoded as a 1-of-K binary indicator matrix $Y$, i.e., $y_{i,k}=1$ if sample $i$ has label $k$ from a set of $K$ labels. Let $P$ be a matrix of probability estimates $p_{i,k}= \mathrm{Pr}(y_{i,k}=1)$. Then the log-loss of the whole set is
$$loss(y_{pred}, y_{true},\mathbf{W}) = - \log \mathrm{Pr}(Y |P)  + \frac{\alpha}{2}\lVert \mathbf{W} \rVert_2^2 = - \frac{1}{N}\sum_{i=0}^{N-1}\sum_{k=0}^{K-1} y_{i,k} \log p_{i.k}  + \frac{\alpha}{2}\lVert \mathbf{W} \rVert_2^2,$$ where $N$ is the number of samples.

In [38]:
print("Loss:", mlp_classifier.loss_)

Loss: 0.0034728684994180608


In [39]:
print("Number of Coefs - the weight matrices:", len(mlp_classifier.coefs_))

[weights.shape for weights in mlp_classifier.coefs_]

Number of Coefs - the weight matrices: 2


[(64, 100), (100, 10)]

In [40]:
print("Number of Intercepts - the number of vectors with biases:", len(mlp_classifier.intercepts_))

[intercept.shape for intercept in mlp_classifier.intercepts_]

Number of Intercepts - the number of vectors with biases: 2


[(100,), (10,)]

In [41]:
print("Number of Iterations for Which Estimator Ran:", mlp_classifier.n_iter_)

Number of Iterations for Which Estimator Ran: 125


In [42]:
print("Name of Output Layer Activation Function:", mlp_regressor.out_activation_)

Name of Output Layer Activation Function: identity


## `GridSearchCV`

Again we can try to optimize the model using GridSearchCV.

In [43]:
%%time

from sklearn.model_selection import GridSearchCV

params = {'activation': ['relu', 'tanh', 'logistic', 'identity'],
          'hidden_layer_sizes': [(100,), (50,100,), (50,75,100,)],
          'solver': ['adam', 'sgd', 'lbfgs'],
          'learning_rate' : ['constant', 'adaptive', 'invscaling']
         }

mlp_classif_grid = GridSearchCV(MLPClassifier(random_state=123), param_grid=params, n_jobs=-1, cv=5, verbose=5)
mlp_classif_grid.fit(X_train,y_train)

print('Train Accuracy : %.3f'%mlp_classif_grid.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%mlp_classif_grid.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%mlp_classif_grid.best_score_)
print('Best Parameters : ',mlp_classif_grid.best_params_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Train Accuracy : 1.000
Test Accuracy : 0.986
Best Accuracy Through Grid Search : 0.980
Best Parameters :  {'activation': 'logistic', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'adam'}
Wall time: 2min 45s


## Plotting Confusion Matrix

In [117]:
plot_confusion_matrix(y_test, mlp_classif_grid.best_estimator_.predict(X_test), 10)